# Portfolio selection

In [1]:
%pip install 'opvious>=0.16.8' yfinance


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Formulation

LaTeX equivalent: https://github.com/opvious/examples/blob/main/sources/portfolio-selection.md

In [2]:
import opvious.modeling as om

class PortfolioSelection(om.Model):
    assets = om.Dimension()
    groups = om.Dimension()
    covariance = om.Parameter.continuous(assets, assets)
    expected_return = om.Parameter.continuous(assets)
    minimum_return = om.Parameter.continuous()
    membership = om.Parameter.indicator(assets, groups)
    minimum_allocation = om.Parameter.unit(groups)
    allocation = om.Variable.unit(assets)
    
    @om.objective
    def minimize_risk(self):
        return om.total(
            self.covariance(l, r) * self.allocation(l) * self.allocation(r)
            for l, r in self.assets * self.assets
        )
    
    @om.constraint
    def expected_return_above_minimum(self):
        yield om.total(self.expected_return(a) * self.allocation(a) for a in self.assets) >= self.minimum_return()
        
    @om.constraint
    def allocation_is_total(self):
        yield self.allocation.total() == 1
        
    @om.constraint
    def group_allocation_above_minimum(self):
        for g in self.groups:
            group_allocation = om.total(self.membership(a, g) * self.allocation(a) for a in self.assets)
            yield group_allocation >= self.minimum_allocation(g)
    

model = PortfolioSelection()
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">PortfolioSelection</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{assets}&: A \\
  \S^d_\mathrm{groups}&: G \\
  \S^p_\mathrm{covariance}&: c \in \mathbb{R}^{A \times A} \\
  \S^p_\mathrm{expectedReturn}&: r^\mathrm{expected} \in \mathbb{R}^{A} \\
  \S^p_\mathrm{minimumReturn}&: r^\mathrm{minimum} \in \mathbb{R} \\
  \S^p_\mathrm{membership}&: m \in \{0, 1\}^{A \times G} \\
  \S^p_\mathrm{minimumAllocation}&: a^\mathrm{minimum} \in [0, 1]^{G} \\
  \S^v_\mathrm{allocation}&: \alpha \in [0, 1]^{A} \\
  \S^o_\mathrm{minimizeRisk}&: \min \sum_{a, a' \in A} c_{a,a'} \alpha_{a} \alpha_{a'} \\
  \S^c_\mathrm{expectedReturnAboveMinimum}&: \sum_{a \in A} r^\mathrm{expected}_{a} \alpha_{a} \geq r^\mathrm{minimum} \\
  \S^c_\mathrm{allocationIsTotal}&: \sum_{a \in A} \alpha_{a} = 1 \\
  \S^c_\mathrm{groupAllocationAboveMinimum}&: \forall g \in G, \sum_{a \in A} m_{a,g} \alpha_{a} \geq a^\mathrm{minimum}_{g} \\
\end{align*}
$$
</div>
</details>
</div>

## Download input data

We gather tickers from Wikipedia and recent performance data via `yfinance`.

In [3]:
import pandas as pd
import yfinance as yf

tickers_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
values_df = yf.download(tickers=list(tickers_df['Symbol'])[:10], start='2022-1-1', interval='1mo')['Adj Close']
returns_df = values_df.head(100).dropna(axis=1).pct_change().dropna(axis=0, how='all')
returns_df.head()

[*********************100%***********************]  10 of 10 completed


,AAP,ABBV,ABT,ACN,ADBE,ADM,ADP,AOS,ATVI,MMM
Date,,,,,,,,,,
2022-02-01,-0.116755,0.090681,-0.050326,-0.103910,-0.124686,0.046000,-0.008391,-0.099266,0.031515,-0.104626
2022-03-01,0.012128,0.097043,-0.018737,0.067116,-0.025787,0.156653,0.112992,-0.068387,-0.017055,0.011229
2022-04-01,-0.028395,-0.093949,-0.041061,-0.109332,-0.130964,-0.007755,-0.036357,-0.085459,-0.056298,-0.031300
2022-05-01,-0.048941,0.011864,0.039015,-0.003367,0.051850,0.014069,0.021817,0.033525,0.036282,0.035154
2022-06-01,-0.088328,0.039289,-0.075004,-0.069724,-0.121062,-0.141624,-0.057863,-0.090486,-0.000257,-0.124403


## Find the optimal allocation

Asset groups are left as exercise to the reader.

In [5]:
import opvious

async def optimal_allocation(returns):
    """Returns an optimal allocation of assets given the input returns"""
    problem = opvious.Problem(
        specification=model.specification(),
        parameters={
            'covariance': returns.cov().stack(),
            'expectedReturn': returns.mean(),
            'minimumReturn': 0.005,
            'membership': {},
            'minimumAllocation': {},
        },
    )
    solution = await opvious.Client.default().solve(problem)
    allocation = solution.outputs.variable('allocation')
    return allocation.reset_index(names=['ticker']).join(
        returns_df.agg(['mean', 'var']).T,
        on='ticker',
        validate='one_to_one'
    ).sort_values(by=['value'], ascending=False)

In [6]:
await optimal_allocation(returns_df)

,ticker,value,dual_value,mean,var
8,ATVI,0.372892,0.000000,0.010304,0.002701
2,ABT,0.247407,0.000000,-0.002287,0.003246
1,ABBV,0.165756,0.000000,0.008706,0.004782
6,ADP,0.091108,0.000000,0.012134,0.004986
3,ACN,0.068455,0.000000,-0.002301,0.006173
5,ADM,0.033382,0.000000,0.011285,0.008353
0,AAP,0.021000,0.000000,-0.049729,0.018583
4,ADBE,0.000000,0.001221,0.006867,0.016534
7,AOS,0.000000,0.000121,0.005759,0.010556
9,MMM,0.000000,0.000162,-0.018532,0.006634
